## 全部入れるよver

In [9]:
# 分析用
import numpy as np
import pandas as pd

# プロット用
import matplotlib.pyplot as plt
%matplotlib inline

# 機械学習用
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit

In [10]:
df = pd.read_csv("train.csv")

In [11]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [12]:
df = df.loc[:, ["PassengerId", "Survived", "Pclass","Fare", "Embarked", "Sex", "Age", "SibSp", "Parch"]]
df = df[np.isnan(df['Age']) == False]

In [13]:
df['Sex'] = df['Sex'].map(lambda x: 0 if x == "male" else 1)
df['Pclass_1'] = df['Pclass'].map(lambda x: 1 if x == 1 else 0)
df['Pclass_2'] = df['Pclass'].map(lambda x: 1 if x == 2 else 0)
df['Embarked_C'] = df['Embarked'].map(lambda x: 1 if x == "C" else 0)
df['Embarked_S'] = df['Embarked'].map(lambda x: 1 if x == "S" else 0)
df = df.drop("Pclass", axis = 1)
df = df.drop("Embarked", axis = 1)

In [14]:
df.head()

,PassengerId,Survived,Fare,Sex,Age,SibSp,Parch,Pclass_1,Pclass_2,Embarked_C,Embarked_S
0,1,0,7.2500,0,22.0,1,0,0,0,0,1
1,2,1,71.2833,1,38.0,1,0,1,0,1,0
2,3,1,7.9250,1,26.0,0,0,0,0,0,1
3,4,1,53.1000,1,35.0,1,0,1,0,0,1
4,5,0,8.0500,0,35.0,0,0,0,0,0,1


In [15]:
#各変数を正規化
df2 = df.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
df2["PassengerId"] = df['PassengerId']
df2['Survived'] = df['Survived']

In [16]:
df2_except = df2.drop("Survived", axis=1).drop("PassengerId", axis=1)
X = df2_except.as_matrix()
Y = df2['Survived'].as_matrix()

In [17]:
clf = LogisticRegression()
clf.fit(X, Y)
df_result = pd.DataFrame({"Name":df2_except.columns,"Coefficients":clf.coef_[0]}).sort_values(by='Coefficients')
df_result.ix[:,['Name', 'Coefficients']]

,Name,Coefficients
2,Age,-2.209895
3,SibSp,-1.100719
4,Parch,-0.245205
8,Embarked_S,0.116392
0,Fare,0.490017
7,Embarked_C,0.569269
6,Pclass_2,1.048238
5,Pclass_1,2.022090
1,Sex,2.445935


In [18]:
clf.intercept_[0]

-0.4879175707980708

In [19]:
clf.score(X,Y)

0.79551820728291311

In [25]:
#予測してみる
df2['eq'] = -0.488 - 2.2099*df2['Age'] + 2.446 * df2['Sex'] + 2.022 * df2['Pclass_1'] + 1.048 * df2['Pclass_2'] - 1.1 * df2['SibSp'] - 0.245 * df2['Parch'] + 0.116 * df2['Embarked_S'] + 0.569 * df2['Embarked_C'] + 0.49 * df2['Fare']

In [26]:
df2['e^eq'] = np.exp(df2['eq'])
df2['Survived-predicted'] = df2['e^eq'] / (1 + df2['e^eq'])
df_pre = df2.ix[:,['Survived', 'Survived-predicted']]
df_pre['prediction'] = df_pre['Survived-predicted'].map(lambda x : 1 if x > 0.5 else 0)

In [27]:
r = {}
r["1-1"] = np.logical_and(df_pre['Survived'] == 1, df_pre['prediction'] == 1).value_counts(sort=False)[1]
r["1-0"] = np.logical_and(df_pre['Survived'] == 1, df_pre['prediction'] == 0).value_counts(sort=False)[1]
r["0-1"] = np.logical_and(df_pre['Survived'] == 0, df_pre['prediction'] == 1).value_counts(sort=False)[1]
r["0-0"] = np.logical_and(df_pre['Survived'] == 0, df_pre['prediction'] == 0).value_counts(sort=False)[1]
r

{'0-0': 362, '0-1': 62, '1-0': 84, '1-1': 206}

In [28]:
#正解率
(r['0-0'] + r['1-1']) / (r['0-0'] + r['1-1'] + r["1-0"] + r["0-1"]) *100

79.551820728291318